<a href="https://colab.research.google.com/github/FabioBoccia/Progetto_ESM/blob/main/Progetto_Bozza_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!%reset -f
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io
from tensorflow import keras